# Introduction

My friend [Andrew Key] and I are developing a series of [Data Science Challenges] to help each other and others become better Data Scientists by working on hands-on projects.

This notebook contains my work for the first challenge, Exploring the Meetup API in my city of choice, beautiful **Vancouver, BC**.

## Challenge 01: Explore the Meetup API

Use the [Meetup API] to explore meetups in your city of choice.

**Guide Questions:**

Below are some guide line questions to get you started:

What is the largest meetup in your location of choice (city, cities, country...etc)?
How many meetups of a certain category (e.g. Tech, Art...etc) are in your city?
Basic statistics of meetups
What is the average size of meetups?
How frequently do meetups host events?

[//]: # (References)
[Andrew Key]: https://github.com/redpanda-ai
[Data Science Challenges]: https://johannesgiorgis.com/introducing-data-science-challenges/
[Meetup API]: https://www.meetup.com/meetup_api/

In [1]:
# import libraries

import datetime
import matplotlib.pyplot as plt
import math
import meetup.api
import os
import pandas as pd
import pprint
import requests

%matplotlib inline

Make sure you have set up **MEETUP_API_KEY** in your environment:

In [2]:
client = meetup.api.Client()

---

## Helper Functions

Let's first create some helper functions.

I got the `parse_response` function from my friend Andrew. Below are the explanations from his [notebook]:

This helper function helps us parse out the two most useful different pieces:

1. **meta**: an object containing meta-data about the response object itself
2. **results**: A page of actual data from the entire result of our API call



[notebook]: https://github.com/redpanda-ai/datascience_challenges/blob/master/01_explore_meetup_api/explore_sf_meetups.ipynb

In [6]:
def parse_response(response):
    '''returns two dataframes, meta and results:
        - meta: a vertically aligned dataframe, where each row is an element
            of the response.meta dictionary
        - results: a horizontally aligned dataframe, where each column is
            an element of the response.results dictionary'''
    meta = pd.DataFrame.from_dict(response.meta, orient='index')
    results = pd.DataFrame.from_dict(response.results)
    return (meta, results)

---

## Categories

When getting Meetup groups via the [GetGroups] method, we need to provide a category id.

Let's find out the different categories and their IDs:

[GetGroups]: https://meetup-api.readthedocs.io/en/latest/meetup_api.html?highlight=getgroups#meetup.api.meetup.api.Client.GetGroups

In [7]:
categories = client.GetCategories()

29/30 (10 seconds remaining)


In [8]:
categories_meta_df, categories_df = parse_response(categories)

In [9]:
categories_meta_df

,0
next,
method,Categories
total_count,33
link,https://api.meetup.com/2/categories
count,33
description,Returns a list of Meetup group categories
lon,None
title,Categories
url,https://api.meetup.com/2/categories?offset=0&f...
id,


We observe there are 33 Meetup categories.

### How the Meetup API works¶

Notice that the value of `next` (above) is an empty string. Meetup API v2 response payloads come in pages, one at a time, but provide the URI of the next API call in the sequence. We can use this to programmatically get each next page in `response.meta["next"]`. until the complete result is returned.

As we can see, the `response.meta["next"]` for this page is an empty string, so all of the categories fit into our first API call.

In [7]:
results = categories.results
print("ID\tName\t\t\t\tShort Name")
for result in results:
    print(f"{result['id']:<5} {result['name']:<30}{result['shortname']}")

ID	Name				Short Name
1     Arts & Culture                Arts
18    Book Clubs                    Book Clubs
2     Career & Business             Business
3     Cars & Motorcycles            Auto
4     Community & Environment       Community
5     Dancing                       Dancing
6     Education & Learning          Education
8     Fashion & Beauty              Fashion
9     Fitness                       Fitness
10    Food & Drink                  Food & Drink
11    Games                         Games
13    Movements & Politics          Movements
14    Health & Wellbeing            Well-being
15    Hobbies & Crafts              Crafts
16    Language & Ethnic Identity    Languages
12    LGBT                          LGBT
17    Lifestyle                     Lifestyle
20    Movies & Film                 Films
21    Music                         Music
22    New Age & Spirituality        Spirituality
23    Outdoors & Adventure          Outdoors
24    Paranormal                    Parano

---

## Explore Cities

In [8]:
cities_resp = client.GetCities(country='Canada', query='Vancouver')

29/30 (10 seconds remaining)


In [9]:
cities_resp.meta

{'next': '',
 'method': 'Cities',
 'total_count': 2,
 'link': 'https://api.meetup.com/2/cities',
 'count': 2,
 'description': "Returns Meetup cities. This method supports search by latitude/longitude/radius, by country/state, by query term/zip, or a combination of all of these. Location-only searches by lat and lon return all cities within a radius of the provided coordinates. Searches with a query return up to 10 cities matching the term, and can be sorted by size or distance to a given coordinate. 'smart' ordering can be used to return the match(es) with the highest member_count, unless a smaller size match exists nearby the given coordinates. Query searches are supported for country but not country and state",
 'lon': 'None',
 'title': 'Cities',
 'url': 'https://api.meetup.com/2/cities?offset=0&query=Vancouver&format=json&page=200&radius=50&key=48f48112b3058e21767448548572a&order=size&desc=false',
 'id': '',
 'updated': 1263132772000,
 'lat': 'None'}

In [10]:
cities = {}

cities_result = cities_resp.results

for result in cities_result:
    city = result['city']
    if city in cities:
        print(f"ERROR: {city} already in dictionary!")
    else:
        cities[result['city']] = result

ERROR: Vancouver already in dictionary!


In [11]:
city_names = list(cities.keys())
sorted(city_names)

['Vancouver']

---

## Explore Groups

In [12]:
# searching for Vancouver, Canda tech groups
group_resp = client.GetGroups(category_id=34, city='Vancouver', country='ca', offset=0)

29/30 (10 seconds remaining)


In [13]:
group_resp.meta

{'next': 'https://api.meetup.com/2/groups?country=ca&offset=1&city=Vancouver&format=json&lon=-123.040000916&category_id=34&page=200&radius=25.0&fields=&key=48f48112b3058e21767448548572a&lat=49.2799987793&order=id&desc=false',
 'method': 'Groups',
 'total_count': 431,
 'link': 'https://api.meetup.com/2/groups',
 'count': 200,
 'description': 'None',
 'lon': -123.04000091552734,
 'title': 'Meetup Groups v2',
 'url': 'https://api.meetup.com/2/groups?country=ca&offset=0&city=Vancouver&format=json&lon=-123.040000916&category_id=34&page=200&radius=25.0&fields=&key=48f48112b3058e21767448548572a&lat=49.2799987793&order=id&desc=false',
 'id': '',
 'updated': 1550467752000,
 'lat': 49.279998779296875}

In [16]:
group_resp.results[:1]

[{'utc_offset': -28800000,
  'country': 'CA',
  'visibility': 'public',
  'city': 'Vancouver',
  'timezone': 'Canada/Pacific',
  'created': 1034097731000,
  'topics': [{'urlkey': 'smallbiz', 'name': 'Small Business', 'id': 389},
   {'urlkey': 'marketing', 'name': 'Marketing', 'id': 1238},
   {'urlkey': 'socialnetwork', 'name': 'Social Networking', 'id': 4422},
   {'urlkey': 'wordpress', 'name': 'WordPress', 'id': 8304},
   {'urlkey': 'creative-writing', 'name': 'Creative Writing', 'id': 15322},
   {'urlkey': 'social-media', 'name': 'Social Media', 'id': 15401},
   {'urlkey': 'online-marketing', 'name': 'Online Marketing', 'id': 15585},
   {'urlkey': 'professional-networking',
    'name': 'Professional Networking',
    'id': 15720},
   {'urlkey': 'writing', 'name': 'Writing', 'id': 16728},
   {'urlkey': 'small-business-marketing-strategy',
    'name': 'Small Business Marketing Strategy',
    'id': 17325},
   {'urlkey': 'social-media-marketing',
    'name': 'Social Media Marketing',
    